In [243]:
import os
import pandas as pd
import numpy as np
from time import time
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, concatenate, Flatten, Dense, Dropout, merge
from keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import keras
import tensorflow as tf

In [244]:
#column headers for the dataset
data_cols = ['user id','movie id','rating','timestamp']
item_cols = ['movie id','movie title','release date',
'video release date','IMDb URL','unknown','Action',
'Adventure','Animation','Childrens','Comedy','Crime',
'Documentary','Drama','Fantasy','Film-Noir','Horror',
'Musical','Mystery','Romance ','Sci-Fi','Thriller',
'War' ,'Western']
user_cols = ['user id','age','gender','occupation',
'zip code']

In [245]:
#importing the data files onto dataframes - 100k
# https://www.kaggle.com/prajitdatta/movielens-100k-dataset
users = pd.read_csv('data/ml-100k/u.user', sep='|',
names=user_cols, encoding='latin-1')
item = pd.read_csv('data/ml-100k/u.item', sep='|',
names=item_cols, encoding='latin-1')
data = pd.read_csv('data/ml-100k/u.data', sep='\t',
names=data_cols, encoding='latin-1')

In [135]:
#importing the data files onto dataframes - 1m
# https://www.kaggle.com/vatsal73/movielens-1m
# users = pd.read_csv('data/ml-1m/users.dat', sep='::', header=None, engine='python', names=user_cols)
# item = pd.read_csv('data/ml-1m/movies.dat', sep='::', header=None, engine='python', names=item_cols)
# data = pd.read_csv('data/ml-1m/ratings.dat', sep='::', header=None, engine='python', names=data_cols)

In [246]:
users.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [247]:
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [248]:
data.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [249]:
# Create a merged dataframe
df = pd.merge(pd.merge(item, data), users)

In [250]:
df.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Thriller,War,Western,user id,rating,timestamp,age,gender,occupation,zip code
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,308,4,887736532,60,M,retired,95076
1,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,308,5,887737890,60,M,retired,95076
2,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,1,0,0,308,4,887739608,60,M,retired,95076
3,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,308,4,887738847,60,M,retired,95076
4,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,308,5,887736696,60,M,retired,95076


### Prepare Wide Input Data

In [251]:
df_wide = df[['gender', 'occupation']] ## Include only small number of cross-product feature transformations

In [252]:
df_wide['gender_occupation'] = df_wide['gender'] + "_" + df_wide['occupation']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [253]:
df_wide.head()

,gender,occupation,gender_occupation
0,M,retired,M_retired
1,M,retired,M_retired
2,M,retired,M_retired
3,M,retired,M_retired
4,M,retired,M_retired


In [254]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(df_wide[['gender_occupation']])

OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [255]:
one_hot_encoded_df = pd.DataFrame(encoder.transform(df_wide[['gender_occupation']]).toarray(), columns=encoder.get_feature_names())

In [256]:
df_wide = df_wide.join(one_hot_encoded_df)

In [257]:
df_wide.drop(['gender', 'occupation', 'gender_occupation'], axis=1, inplace=True)

In [258]:
df_wide.head()

,x0_F_administrator,x0_F_artist,x0_F_educator,x0_F_engineer,x0_F_entertainment,x0_F_executive,x0_F_healthcare,x0_F_homemaker,x0_F_lawyer,x0_F_librarian,...,x0_M_marketing,x0_M_none,x0_M_other,x0_M_programmer,x0_M_retired,x0_M_salesman,x0_M_scientist,x0_M_student,x0_M_technician,x0_M_writer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [259]:
df_wide.shape

(100000, 41)

### Prepare Deep Input Data

In [260]:
df_deep = df[['age', 'unknown', 'Action', 'Adventure', 'Animation', 'Childrens',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
              'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci-Fi', 'Thriller', 'War',
              'Western', 'gender', 'occupation']]

In [261]:
df_deep.head()

,age,unknown,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,gender,occupation
0,60,0,0,0,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,M,retired
1,60,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,M,retired
2,60,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,M,retired
3,60,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,M,retired
4,60,0,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,M,retired


In [262]:
# Combine sparse categorical features into one single genre feature
df_deep['genre'] = df_deep[['unknown', 'Action', 'Adventure', 'Animation', 'Childrens',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
              'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci-Fi', 'Thriller', 'War',
              'Western']].idxmax(1)
df_deep.drop(columns=['unknown', 'Action', 'Adventure', 'Animation', 'Childrens',
              'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
              'Horror', 'Musical', 'Mystery', 'Romance ', 'Sci-Fi', 'Thriller', 'War',
              'Western'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [264]:
df_deep.head()

,age,gender,occupation,genre
0,60,M,retired,Animation
1,60,M,retired,Action
2,60,M,retired,Crime
3,60,M,retired,Drama
4,60,M,retired,Childrens


In [279]:
# Encode categorical features
for feature in ['gender', 'occupation', 'genre']:
    encoder = LabelEncoder()
    encoder.fit(df_deep[[feature]])
    transformed_feature = encoder.transform(df_deep[[feature]])
    df_deep[feature] = transformed_feature
    # Todo: save all encoders in a dict, save the dict.
print(df_deep.head())

       age  gender  occupation  genre
0  0.80303       1          15      2
1  0.80303       1          15      0
2  0.80303       1          15      5
3  0.80303       1          15      7
4  0.80303       1          15      3


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [280]:
# Min-max scaling for numerical features
for feature in ['age']:
    scaler = MinMaxScaler()
    scaler.fit(df_deep[[feature]])
    transformed_feature = scaler.transform(df_deep[[feature]])
    df_deep[feature] = transformed_feature
    # Todo: save all scaler in a dict, save the dict
print(df_deep.head())

       age  gender  occupation  genre
0  0.80303       1          15      2
1  0.80303       1          15      0
2  0.80303       1          15      5
3  0.80303       1          15      7
4  0.80303       1          15      3


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [266]:
df_deep.shape

(100000, 4)

### Define Wide and Deep Model Architecture

In [267]:
# Define our deep network with the Functional API
max_num_categorical_values = 50
emb_dimension = 64
max_sequence_length = 1

categorical_input_1 = Input(shape=(1,), name='categorical_input_1')
emb_1 = Embedding(input_dim=max_num_categorical_values, output_dim=emb_dimension,
                  input_length=max_sequence_length, name='emb_1')(categorical_input_1)
emb_1 = Flatten()(emb_1)

categorical_input_2 = Input(shape=(1,), name='categorical_input_2')
emb_2 = Embedding(input_dim=max_num_categorical_values, output_dim=emb_dimension,
                  input_length=max_sequence_length, name='emb_2')(categorical_input_2)
emb_2 = Flatten()(emb_2)

categorical_input_3 = Input(shape=(1,), name='categorical_input_3')
emb_3 = Embedding(input_dim=max_num_categorical_values, output_dim=emb_dimension,
                  input_length=max_sequence_length, name='emb_3')(categorical_input_3)
emb_3 = Flatten()(emb_3)

numerical_input = Input(shape=(1,), name='numerical_input')

concatenated_embeddings = concatenate([emb_1, emb_2, emb_3, numerical_input])

concatenated_embeddings = Dropout(rate=0.2)(concatenated_embeddings)
x1 = Dense(64, activation='relu')(concatenated_embeddings)
x1 = Dropout(rate=0.2)(x1)

x2 = Dense(64, activation='relu')(x1)
x2 = Dropout(rate=0.2)(x2)

x3 = Dense(64, activation='relu')(x2)
x3 = Dropout(rate=0.2)(x3)

x4 = Dense(64, activation='relu')(merge.add([x1, x3]))
x4 = Dropout(rate=0.2)(x4)

x5 = Dense(64, activation='relu')(x4)
x5 = Dropout(rate=0.2)(x5)

x6 = Dense(64, activation='relu')(x5)
x6 = Dropout(rate=0.2)(x6)

x7 = Dense(64, activation='relu')(merge.add([x4, x6]))
x7 = Dropout(rate=0.2)(x7)

x8 = Dense(64, activation='relu')(x7)
x8 = Dropout(rate=0.2)(x8)

x9 = Dense(64, activation='relu')(x8)
x9 = Dropout(rate=0.2)(x9)

deep_output = Dense(64, activation='relu')(x9)

In [268]:
# Define our wide network with the Functional API
num_features = len(df_wide.columns)
wide_inputs = Input(shape=(num_features,), name='wide_inputs')

In [269]:
# Combine wide and deep into one model
x = concatenate([wide_inputs, deep_output])
x = Dropout(rate=0.2)(x)
wide_and_deep_output = Dense(1, activation='relu')(x)
wide_and_deep_model = Model(inputs=[wide_inputs] + [categorical_input_1] + 
                            [categorical_input_2] + [categorical_input_3] + 
                            [numerical_input], outputs=wide_and_deep_output)
wide_and_deep_model.summary()
plot_model(wide_and_deep_model, to_file='wide_and_deep_model.png', show_shapes=True, show_layer_names=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
categorical_input_1 (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
categorical_input_2 (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
categorical_input_3 (InputLayer (None, 1)            0                                            
__________________________________________________________________________________________________
emb_1 (Embedding)               (None, 1, 64)        3200        categorical_input_1[0][0]        
__________________________________________________________________________________________________
emb_2 (Emb

### Prepare data for Train and Test

In [281]:
# Split data
X = pd.concat([df_wide, df_deep], axis=1)
y = df[['rating']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [282]:
def prepare_independent_variables(X):
    """Split X dataframe into its separate input components for the neural network
    """
    wide_inputs = X[['x0_F_administrator', 'x0_F_artist', 'x0_F_educator', 'x0_F_engineer',
                     'x0_F_entertainment', 'x0_F_executive', 'x0_F_healthcare',
                     'x0_F_homemaker', 'x0_F_lawyer', 'x0_F_librarian', 'x0_F_marketing',
                     'x0_F_none', 'x0_F_other', 'x0_F_programmer', 'x0_F_retired',
                     'x0_F_salesman', 'x0_F_scientist', 'x0_F_student', 'x0_F_technician',
                     'x0_F_writer', 'x0_M_administrator', 'x0_M_artist', 'x0_M_doctor',
                     'x0_M_educator', 'x0_M_engineer', 'x0_M_entertainment',
                     'x0_M_executive', 'x0_M_healthcare', 'x0_M_homemaker', 'x0_M_lawyer',
                     'x0_M_librarian', 'x0_M_marketing', 'x0_M_none', 'x0_M_other',
                     'x0_M_programmer', 'x0_M_retired', 'x0_M_salesman', 'x0_M_scientist',
                     'x0_M_student', 'x0_M_technician', 'x0_M_writer']].values
    categorical_input_1 = X[['gender']].values
    categorical_input_2 = X[['occupation']].values
    categorical_input_3 = X[['genre']].values
    numerical_input = X[['age']].values
        
    return wide_inputs, categorical_input_1, categorical_input_2, categorical_input_3, \
           numerical_input

In [283]:
wide_inputs_train, categorical_input_1_train, categorical_input_2_train, \
categorical_input_3_train, numerical_input_train = prepare_independent_variables(X_train)

wide_inputs_test, categorical_input_1_test, categorical_input_2_test, \
categorical_input_3_test, numerical_input_test = prepare_independent_variables(X_test)

y_train = y_train.values
y_test = y_test.values

### Compile, Train, Evaluate

In [284]:
wide_and_deep_model.compile(loss='mse',
                            optimizer='adam',
                            metrics=['mse'])

In [287]:
keras.backend.get_session().run(tf.global_variables_initializer())

In [288]:
date_time = (datetime.utcnow() + timedelta(hours=8)).strftime('[%Y-%m-%d %H-%M-%S]')
tensorboard = TensorBoard(log_dir='./logs/tensorboard/{}'.format(date_time))

early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint(filepath='./model/wide_and_deep_model.h5', 
                                   monitor='val_loss',
                                   save_weights_only=True, 
                                   save_best_only=True)
callbacks = [model_checkpoint,early_stopping, tensorboard]

wide_and_deep_model.fit(x={'wide_inputs': wide_inputs_train,
                           'categorical_input_1': categorical_input_1_train,
                           'categorical_input_2': categorical_input_2_train,
                           'categorical_input_3': categorical_input_3_train,
                           'numerical_input': numerical_input_train},
                        y=y_train,
                        batch_size=32, epochs=500, verbose=1,
                        callbacks=callbacks, validation_split=0.2)

Train on 64000 samples, validate on 16000 samples
Epoch 1/500
64000/64000 [==============================] - 9s 134us/step - loss: 1.4997 - mean_squared_error: 1.4997 - val_loss: 1.2277 - val_mean_squared_error: 1.2277
Epoch 2/500
64000/64000 [==============================] - 8s 119us/step - loss: 1.3102 - mean_squared_error: 1.3102 - val_loss: 1.2260 - val_mean_squared_error: 1.2260
Epoch 3/500
64000/64000 [==============================] - 8s 132us/step - loss: 1.2877 - mean_squared_error: 1.2877 - val_loss: 1.2217 - val_mean_squared_error: 1.2217
Epoch 4/500
64000/64000 [==============================] - 9s 135us/step - loss: 1.2731 - mean_squared_error: 1.2731 - val_loss: 1.2236 - val_mean_squared_error: 1.2236
Epoch 5/500
64000/64000 [==============================] - 8s 130us/step - loss: 1.2681 - mean_squared_error: 1.2681 - val_loss: 1.2256 - val_mean_squared_error: 1.2256
Epoch 6/500
64000/64000 [==============================] - 8s 121us/step - loss: 1.2620 - mean_squared_er

In [289]:
wide_and_deep_model.evaluate(x={'wide_inputs': wide_inputs_test,
                                'categorical_input_1': categorical_input_1_test,
                                'categorical_input_2': categorical_input_2_test,
                                'categorical_input_3': categorical_input_3_test,
                                'numerical_input': numerical_input_test},
                             y=y_test,
                             batch_size=32, verbose=1)

20000/20000 [==============================] - 1s 41us/step


[1.2104825955867768, 1.2104825955867768]

In [290]:
wide_and_deep_model.metrics_names

['loss', 'mean_squared_error']

In [291]:
# To view tensorboard, input following in terminal:
# tensorboard --logdir=logs/tensorboard
# Navigate to http://localhost:6006

## Inference

In [292]:
predictions = wide_and_deep_model.predict(x={'wide_inputs': wide_inputs_test,
                                             'categorical_input_1': categorical_input_1_test,
                                             'categorical_input_2': categorical_input_2_test,
                                             'categorical_input_3': categorical_input_3_test,
                                             'numerical_input': numerical_input_test},
                                          batch_size=32, verbose=1)

20000/20000 [==============================] - 1s 74us/step
